In [1]:
import ixmp
import pyam
import nomenclature
import yaml

<IPython.core.display.Javascript object>

In [17]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [70]:
eu_climate_dsd = nomenclature.DataStructureDefinition("../definitions/")
eu27_processor = nomenclature.RegionProcessor.from_directory("../mappings_eu27/")

eu27_uk_countries = (
    eu27_processor.mappings["MESSAGEix-GLOBIOM 1.1-M-R12"]
    .common_regions[1]
    .constituent_regions
)

In [4]:
ngfs_public = ixmp.Platform("ngfs-phase3")

In [5]:
ngs_public_scenarios = ngfs_public.scenario_list()

In [10]:
list(ngs_public_scenarios.model.unique())

['Downscaling [GCAM 5.3+ NGFS]',
 'Downscaling [MESSAGEix-GLOBIOM 1.1-M-R12]',
 'Downscaling [REMIND-MAgPIE 3.0-4.4]',
 'GCAM 5.3+ NGFS',
 'MESSAGEix-GLOBIOM 1.1-M-R12',
 'NiGEM NGFS v1.22',
 'NiGEM NGFS v1.22 [GCAM 5.3+ NGFS]',
 'NiGEM NGFS v1.22 [MESSAGEix-GLOBIOM 1.1-M-R12]',
 'NiGEM NGFS v1.22 [REMIND-MAgPIE 3.0-4.4]',
 'REMIND-MAgPIE 3.0-4.4',
 'REMIND-MAgPIE 3.0-4.4 IntegratedPhysicalDamages (95th-high)',
 'REMIND-MAgPIE 3.0-4.4 IntegratedPhysicalDamages (median)']

In [7]:
ngs_public_scenarios.scenario.unique()

array(['Below 2°C', 'Current Policies', 'Delayed transition',
       'Divergent Net Zero', 'Nationally Determined Contributions (NDCs)',
       'Net Zero 2050', 'Baseline'], dtype=object)

In [93]:
model = "GCAM 5.3+ NGFS"
ngs_public_scenarios[ngs_public_scenarios.model == model]
scenarios = ngs_public_scenarios[ngs_public_scenarios.model == model].scenario.unique()

In [94]:
scenarios

array(['Below 2°C', 'Current Policies', 'Delayed transition',
       'Divergent Net Zero', 'Nationally Determined Contributions (NDCs)',
       'Net Zero 2050'], dtype=object)

In [97]:
for s in scenarios:
    scen = ixmp.Scenario(ngfs_public, model=model, scenario=s)
    df_native = pyam.IamDataFrame(scen.timeseries())

    scen = ixmp.Scenario(ngfs_public, model=f"Downscaling [{model}]", scenario=s)
    df_downscaled = pyam.IamDataFrame(scen.timeseries())
    df_downscaled.rename(model={f"Downscaling [{model}]": model}, inplace=True)
    df_downscaled.filter(region=eu27_uk_countries, inplace=True)

    df_downscaled_eu = eu27_processor.apply(df_downscaled, eu_climate_dsd)
    
    df = pyam.concat([df_native, df_downscaled, df_downscaled_eu])
    
    df.filter(variable=eu_climate_dsd.variable, inplace=True)

    # MESSAGE 
    df.filter(variable=["Trade"], keep=False, inplace=True)
    df.filter(unit="Index (2005 = 1)", keep=False, inplace=True)

    # REMIND
    remind_rename_mapping = {
        'REMIND-MAgPIE 3.0-4.4|Canada, NZ, Australia':
            'REMIND 3.0|Canada, Australia, New Zealand',
        'REMIND-MAgPIE 3.0-4.4|China':
            'REMIND 3.0|China and Taiwan',
        'REMIND-MAgPIE 3.0-4.4|Countries from the Reforming Economies of the Former Soviet Union':
            'REMIND 3.0|Russia and Reforming Economies',
        'REMIND-MAgPIE 3.0-4.4|EU 28':
            'REMIND 3.0|EU 28',
        'REMIND-MAgPIE 3.0-4.4|India':
            'REMIND 3.0|India',
        'REMIND-MAgPIE 3.0-4.4|Japan':
            'REMIND 3.0|Japan',
        'REMIND-MAgPIE 3.0-4.4|Latin America and the Caribbean':
            'REMIND 3.0|Latin America and the Caribbean',
        'REMIND-MAgPIE 3.0-4.4|Middle East, North Africa, Central Asia':
            'REMIND 3.0|Middle East and North Africa',
        'REMIND-MAgPIE 3.0-4.4|Non-EU28 Europe':
            'REMIND 3.0|Non-EU28 Europe',
        'REMIND-MAgPIE 3.0-4.4|Other Asia':
            'REMIND 3.0|Other Asia',
        'REMIND-MAgPIE 3.0-4.4|Sub-saharan Africa':
            'REMIND 3.0|Sub-Saharan Africa',
        'REMIND-MAgPIE 3.0-4.4|United States of America':
            'REMIND 3.0|United States of America',
    }
    df.rename(region=remind_rename_mapping, inplace=True)

    df.rename(model={
        "REMIND-MAgPIE 3.0-4.4 IntegratedPhysicalDamages (95th-high)":
            "REMIND-MAgPIE 3.0-4.4 w/ Integrated Physical Damages (95th-high)",
        "REMIND-MAgPIE 3.0-4.4 IntegratedPhysicalDamages (median)":
            "REMIND-MAgPIE 3.0-4.4 w/ Integrated Physical Damages (median)"
    }, inplace=True)

    # GCAM
    gcam_region_rename_mapping = dict([
        (i, i.replace("GCAM 5.3+ NGFS", "GCAM 5.3")) for i in df.region if "GCAM 5.3" in i
    ])

    df.rename(region={
        "GCAM 5.3|Australia_NZ": "GCAM 5.3|Australia and New Zealand",
        "GCAM 5.3|EU-12": "GCAM 5.3|Eastern EU",
        "GCAM 5.3|EU-15": "GCAM 5.3|Western EU & UK"
    }, inplace=True)
    df.rename(region=gcam_region_rename_mapping, inplace=True)

    # downselect to known regions
    df.filter(region=eu_climate_dsd.region, inplace=True)

    eu_climate_dsd.validate(df)

    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, df.model[0], s, version="new")
    scen.add_timeseries(df.data)
    scen.commit(f"Migrated from the database of the NGFS Phase 3 scenario ensemble")
    scen.set_as_default()

    scen.set_meta({"Source": "NGFS Phase 3"})

    print(f"Model {model} - Scenario {s}" )

print("Done!")   

2023-01-09 18:45:48 INFO     Applying region-processing for model GCAM 5.3+ NGFS from file ..\mappings_eu27\downscaled_eu27.yaml
2023-01-09 18:45:48 INFO     Could not aggregate 'Price|Agriculture|Corn|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:45:48 INFO     Could not aggregate 'Price|Agriculture|Non-Energy Crops|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:45:48 INFO     Could not aggregate 'Price|Agriculture|Soybean|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:45:48 INFO     Could not aggregate 'Price|Agriculture|Wheat|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:45:48 INFO     Could not aggregate 'Price|Carbon' for region 'EU27' ({'weight': 'Emissions|CO2'})
2023-01-09 18:45:48 INFO     Could not aggregate 'Price|Final Energy|Residential and Commercial|Residential|Electricity' fo

Model GCAM 5.3+ NGFS - Scenario Below 2°C


2023-01-09 18:47:14 INFO     Applying region-processing for model GCAM 5.3+ NGFS from file ..\mappings_eu27\downscaled_eu27.yaml
2023-01-09 18:47:14 INFO     Could not aggregate 'Price|Agriculture|Corn|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:47:14 INFO     Could not aggregate 'Price|Agriculture|Non-Energy Crops|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:47:14 INFO     Could not aggregate 'Price|Agriculture|Soybean|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:47:14 INFO     Could not aggregate 'Price|Agriculture|Wheat|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:47:14 INFO     Could not aggregate 'Price|Carbon' for region 'EU27' ({'weight': 'Emissions|CO2'})
2023-01-09 18:47:14 INFO     Could not aggregate 'Price|Final Energy|Residential and Commercial|Residential|Electricity' fo

Model GCAM 5.3+ NGFS - Scenario Current Policies


2023-01-09 18:48:41 INFO     Applying region-processing for model GCAM 5.3+ NGFS from file ..\mappings_eu27\downscaled_eu27.yaml
2023-01-09 18:48:42 INFO     Could not aggregate 'Price|Agriculture|Corn|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:48:42 INFO     Could not aggregate 'Price|Agriculture|Non-Energy Crops|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:48:42 INFO     Could not aggregate 'Price|Agriculture|Soybean|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:48:42 INFO     Could not aggregate 'Price|Agriculture|Wheat|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:48:42 INFO     Could not aggregate 'Price|Carbon' for region 'EU27' ({'weight': 'Emissions|CO2'})
2023-01-09 18:48:42 INFO     Could not aggregate 'Price|Final Energy|Residential and Commercial|Residential|Electricity' fo

...

Model GCAM 5.3+ NGFS - Scenario Divergent Net Zero


2023-01-09 18:51:34 INFO     Applying region-processing for model GCAM 5.3+ NGFS from file ..\mappings_eu27\downscaled_eu27.yaml
2023-01-09 18:51:35 INFO     Could not aggregate 'Price|Agriculture|Corn|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:51:35 INFO     Could not aggregate 'Price|Agriculture|Non-Energy Crops|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:51:35 INFO     Could not aggregate 'Price|Agriculture|Soybean|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:51:35 INFO     Could not aggregate 'Price|Agriculture|Wheat|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:51:35 INFO     Could not aggregate 'Price|Carbon' for region 'EU27' ({'weight': 'Emissions|CO2'})
2023-01-09 18:51:35 INFO     Could not aggregate 'Price|Final Energy|Residential and Commercial|Residential|Electricity' fo

Model GCAM 5.3+ NGFS - Scenario Nationally Determined Contributions (NDCs)


2023-01-09 18:53:01 INFO     Applying region-processing for model GCAM 5.3+ NGFS from file ..\mappings_eu27\downscaled_eu27.yaml
2023-01-09 18:53:01 INFO     Could not aggregate 'Price|Agriculture|Corn|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:53:01 INFO     Could not aggregate 'Price|Agriculture|Non-Energy Crops|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:53:01 INFO     Could not aggregate 'Price|Agriculture|Soybean|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:53:01 INFO     Could not aggregate 'Price|Agriculture|Wheat|Index' for region 'EU27' ({'weight': 'Agricultural Production|Non-Energy|Crops'})
2023-01-09 18:53:01 INFO     Could not aggregate 'Price|Carbon' for region 'EU27' ({'weight': 'Emissions|CO2'})
2023-01-09 18:53:01 INFO     Could not aggregate 'Price|Final Energy|Residential and Commercial|Residential|Electricity' fo

Model GCAM 5.3+ NGFS - Scenario Net Zero 2050
Done!


In [96]:
df.region

['Argentina',
 'Asia (R5)',
 'Austria',
 'Belgium',
 'Bulgaria',
 'Canada',
 'Colombia',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'EU',
 'EU27',
 'EU27 & UK',
 'Estonia',
 'Federative Republic of Brazil',
 'Finland',
 'France',
 'GCAM 5.3|Africa_Eastern',
 'GCAM 5.3|Africa_Northern',
 'GCAM 5.3|Africa_Southern',
 'GCAM 5.3|Africa_Western',
 'GCAM 5.3|Argentina',
 'GCAM 5.3|Australia and New Zealand',
 'GCAM 5.3|Brazil',
 'GCAM 5.3|Canada',
 'GCAM 5.3|Central America and Caribbean',
 'GCAM 5.3|Central Asia',
 'GCAM 5.3|China',
 'GCAM 5.3|Colombia',
 'GCAM 5.3|Eastern EU',
 'GCAM 5.3|Western EU & UK',
 'GCAM 5.3|Europe_Eastern',
 'GCAM 5.3|Europe_Non_EU',
 'GCAM 5.3|European Free Trade Association',
 'GCAM 5.3|India',
 'GCAM 5.3|Indonesia',
 'GCAM 5.3|Japan',
 'GCAM 5.3|Mexico',
 'GCAM 5.3|Middle East',
 'GCAM 5.3|Pakistan',
 'GCAM 5.3|Russia',
 'GCAM 5.3|South Africa',
 'GCAM 5.3|South America_Northern',
 'GCAM 5.3|South America_Southern',
 'GCAM 5.3|South Asia',
 'GCAM 5.

In [ ]:
x = pyam.IamDataFrame(scen.timeseries())

In [ ]:
x = pyam.IamDataFrame(
    ixmp.Scenario(mp=eu_climate, model="REMIND-MAgPIE 3.0-4.4", scenario="Below 2°C")
    .timeseries()
)

In [ ]:
x.region

In [ ]:
# aggregate from countries to 

eu27_uk_countries = (
    eu27_processor.mappings["MESSAGEix-GLOBIOM 1.1-M-R12"]
    .common_regions[1]
    .constituent_regions
)

df.append(eu27_processor.apply(df.filter(region=eu27_uk_countries), eu_climate_dsd), inplace=True)

In [ ]:
df.filter(region=eu27_uk_countries)

In [ ]:
eu27_processor.apply(df.filter(region=eu27_uk_countries), eu_climate_dsd)

In [ ]:
df.model

In [ ]:
eu27_processor.mappings.keys()

In [ ]:
for s in scenarios:
    scen = ixmp.Scenario(ngfs_public, model=model, scenario=s)
    df = pyam.IamDataFrame(scen.timeseries())

    df.filter(region=eu_climate_dsd.region, inplace=True)

    # MESSAGE 
    df.rename(region={"MESSAGEix-GLOBIOM 1.1-R12|Sub-saharan Africa": "MESSAGEix-GLOBIOM 1.1-R12|Sub-Saharan Africa"}, inplace=True)


    df.filter(region=["World", "MESSAGE*", "* (R5)"] + eur_downscaled, inplace=True)
    df.filter(variable=eu_climate_dsd.variable, inplace=True)
              
    eu27_uk_countries = (
        eu27_processor.mappings["MESSAGEix-GLOBIOM 1.1"]
        .common_regions[1]
        .constituent_regions
    )

    df.append(eu27_processor.apply(df.filter(region=eu27_uk_countries), eu_climate_dsd), inplace=True)
              
    df.rename(unit={
        "Mt NOx/yr": "Mt NO2/yr",
        "billion US$2005/yr": "billion US$2010/yr",
    }, inplace=True)
              
    for v in [i for i in df.variable if i.startswith("Capacity Additions|Electricity")]:
        df.rename(variable={v: v}, unit={"GW": "GW/yr"}, inplace=True)
    
    df.filter(variable=[
        "Capacity Additions|Electricity|Storage Capacity", "Price|Agriculture|Non-Energy Crops and Livestock|Index", "Price|Agriculture|Non-Energy Crops|Index"
    ], keep=False, inplace=True)
              
    eu_climate_dsd.validate(df)
    
    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, df.model[0], s, version="new")
    scen.add_timeseries(df.data)
    scen.commit(f"Migrated from the database of the NGFS Phase 3 scenario ensemble")
    scen.set_as_default()

    scen.set_meta({"Source": "NGFS Phase 3"})
    
    print(f"Model {model_public} - Scenario {s}" )

print("Done!")        

# Development section

In [ ]:
df = pyam.IamDataFrame(
        ixmp.Scenario(mp=ngfs_public, model="MESSAGEix-GLOBIOM 1.1", scenario="Current Policies ").timeseries()
    )

In [ ]:
eur_downscaled = []

for code, attrs in eu_climate_dsd.region.items():
    if attrs.hierarchy == "country":
        d_code = f"{code} (downscaled)"
        #eu_climate.add_region(d_code, hierarchy="country (downscaled)")
        eur_downscaled.append(d_code)

In [ ]:
df.rename(region=region_rename_mapping, inplace=True)

# MESSAGE 
df.rename(region={"MESSAGEix-GLOBIOM 1.1|Sub-saharan Africa": "MESSAGEix-GLOBIOM 1.1|Sub-Saharan Africa"}, inplace=True)


In [ ]:
list()

In [ ]:
df.filter(region=["World", "MESSAGE*", "* (R5)"] + eur_downscaled, inplace=True)

In [ ]:
df.filter(variable=eu_climate_dsd.variable, inplace=True)

In [ ]:
eu27_processor = nomenclature.RegionProcessor.from_directory("../mappings_eu27/")

In [ ]:
eu27_uk_countries = (
    eu27_processor.mappings["MESSAGEix-GLOBIOM 1.1"]
    .common_regions[1]
    .constituent_regions
)
eu27_df = df.filter(region=eu27_uk_countries)

In [ ]:
df_eu27 = eu27_processor.apply(eu27_df, eu_climate_dsd)

In [ ]:
df.append(df_eu27, inplace=True)

In [ ]:
df.rename(unit={
    "Mt NOx/yr": "Mt NO2/yr",
    "billion US$2005/yr": "billion US$2010/yr",
}, inplace=True)

In [ ]:
for v in [i for i in df.variable if i.startswith("Capacity Additions|Electricity")]:
    df.rename(variable={v: v}, unit={"GW": "GW/yr"}, inplace=True)

In [ ]:
df.filter(variable=[
    "Capacity Additions|Electricity|Storage Capacity", "Price|Agriculture|Non-Energy Crops and Livestock|Index", "Price|Agriculture|Non-Energy Crops|Index"
], keep=False, inplace=True)

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
scenario_selection = [[(m, s) for s in scenario_mapping[m]] for m in scenario_mapping.keys()][0]

In [ ]:
scenario_selection

In [ ]:
scenario_list = [
    pyam.IamDataFrame(
        ixmp.Scenario(mp=source_db, model=m, scenario=s).timeseries()
    )
    for (m, s) in scenario_selection
]
df = pyam.concat(scenario_list)

In [ ]:
df.region

In [ ]:
remind_mapping = {}
for key, value in eu_climate_rp.mappings["REMIND 2.1"].rename_mapping.items():
    remind_mapping[f"REMIND 2.1|{key}"] = value

In [ ]:
image_mapping = {}
for key, value in eu_climate_rp.mappings["IMAGE 3.2"].rename_mapping.items():
    remind_mapping[f"IMAGE 3.2|{key}"] = value

In [ ]:
df.rename(region=remind_mapping, inplace=True)

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)

    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, _model, _scenario, version="new")
    scen.add_timeseries(_df.data)
    scen.commit(f"Migrated from H2020 ECEMF database")
    scen.set_as_default()

    scen.set_meta({"Source": "H2020 ECEMF"})